In [ ]:
import pandas as pd


data = pd.read_csv((r'C:\Users\admin\Desktop\TFG\data\dades - LFL.csv'))
data.head()


,Player,Position,Games,Win rate,KDA,Avg kills,Avg deaths,Avg assists,CSM,GPM,...,Avg WCPM,Avg VWPM,GD@15,CSD@15,XPD@15,FB %,FB Victim,Penta Kills,Solo Kills,Punctuation
0,113,JUNGLE,47,66%,3.4,3.0,3.9,10.2,5.5,349,...,0.30,0.24,242,0,2,36.2%,14.9%,0,6,4
1,Abbedagge,MID,25,76%,5.9,4.8,2.1,7.4,9.0,448,...,0.31,0.22,332,6,320,20%,0%,0,6,3
2,Advienne,SUPPORT,10,50%,2.7,1.0,3.6,8.6,0.8,237,...,0.44,0.45,-263,-6,-212,10%,10%,0,-,2
3,Agresivoo,TOP,18,16.7%,1.5,1.4,3.1,3.4,8.1,359,...,0.14,0.16,-595,-8,-121,5.6%,16.7%,1,5,2
4,BAO,ADC,47,40.4%,5.5,4.7,2.0,6.0,9.2,438,...,0.40,0.13,-84,-1,-170,29.8%,6.4%,0,6,2


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim

# Convert percentage fields to numeric
data['Win rate'] = data['Win rate'].str.rstrip('%').astype('float') / 100
data['KP%'] = data['KP%'].str.rstrip('%').astype('float') / 100
data['FB %'] = data['FB %'].str.rstrip('%').astype('float') / 100
data['FB Victim'] = data['FB Victim'].str.rstrip('%').astype('float') / 100
data['Solo Kills'] = data['Solo Kills'].replace('-', 0).astype(float)
data['Solo Kills'] = data['Solo Kills'].astype(float)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim

label_encoders = {}
for col in ['Player']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le


# Separate the data by Punctuation values
punctuation_4_5 = data[data['Punctuation'].isin([4, 5])]
punctuation_1_2_3 = data[data['Punctuation'].isin([1, 2, 3])]

# Perform random oversampling for Punctuation 4 and 5
# We'll sample with replacement to create duplicates of the rows with Punctuation 4 or 5
oversampled_punctuation_4_5 = punctuation_4_5.sample(n=len(punctuation_1_2_3), replace=True, random_state=42)

# Combine back with Punctuation 1, 2, 3 data
balanced_data = pd.concat([punctuation_1_2_3, oversampled_punctuation_4_5])

# Shuffle the balanced dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Define features and target from the balanced dataset
X_balanced = balanced_data.drop(columns=['Punctuation'])
y_balanced = balanced_data['Punctuation']

# Standardize numeric features
scaler = StandardScaler()
X_balanced = scaler.fit_transform(X_balanced)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Define the neural network
class PunctuationPredictor(nn.Module):
    def __init__(self, input_size):
        super(PunctuationPredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Output layer for regression

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model, loss function, and optimizer
model = PunctuationPredictor(input_size=X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_loss = criterion(test_predictions, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

# Display sample predictions with corresponding rows
test_predictions_np = test_predictions.view(-1).numpy()
y_test_np = y_test_tensor.view(-1).numpy()

print("\nSample Predictions with Rows:")
for i in range(10):  # Show first 10 predictions along with row details
    # Get the index of the row in the test set
    row_index = X_test[i]

    # Print the row details along with the predicted and actual values
    print(f"Row {i + 1}: {data.iloc[i]}")  # Print full row from the original dataset
    print(f"Predicted: {test_predictions_np[i]:.2f}, Actual: {y_test_np[i]:.2f}\n")

Epoch 10/50, Loss: 8.8122
Epoch 20/50, Loss: 6.7075
Epoch 30/50, Loss: 4.2967
Epoch 40/50, Loss: 2.1239
Epoch 50/50, Loss: 1.2142
Test Loss: 1.4114

Sample Predictions with Rows:
Row 1: Player                  0
Games                  47
Win rate             0.66
KDA                   3.4
Avg kills             3.0
Avg deaths            3.9
Avg assists          10.2
CSM                   5.5
GPM                   349
KP%                 0.754
DMG%                 16.3
DPM                   478
VSPM                 1.23
Avg WPM              0.27
Avg WCPM              0.3
Avg VWPM             0.24
GD@15                 242
CSD@15                  0
XPD@15                  2
FB %                0.362
FB Victim           0.149
Penta Kills             0
Solo Kills            6.0
Punctuation             4
Position_JUNGLE      True
Position_MID        False
Position_SUPPORT    False
Position_TOP        False
Name: 0, dtype: object
Predicted: 3.25, Actual: 1.00

Row 2: Player                  1

In [ ]:
# Retrieve the original player names
player_encoder = label_encoders['Player']  # Access the saved LabelEncoder for Player

model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_loss = criterion(test_predictions, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

    # Get predictions and actual values
    predictions = test_predictions.view(-1).numpy()
    actuals = y_test_tensor.view(-1).numpy()

    # Retrieve original rows from the dataset using the test indices
    test_indices = y_test.index  # Get original indices from the split
    print("\nFull information for the predicted rows with player names:")
    for i in range(15):  # Show details for the first 10 predictions
        index = test_indices[i]
        row = data.iloc[index]

        # Decode the player name
        original_player_name = player_encoder.inverse_transform([int(row['Player'])])[0]

        print(f"\nRow {index} (Player: {original_player_name}):")
        print(row)  # Print full row details
        print(f"Predicted: {predictions[i]:.2f}, Actual: {actuals[i]:.2f}")

Test Loss: 1.4114

Full information for the predicted rows with player names:


IndexError: single positional indexer is out-of-bounds